In [38]:
#!/usr/bin/env python3

import sys
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Conv1D, concatenate, SpatialDropout1D, GlobalMaxPooling1D
from keras_contrib.layers import CRF

import matplotlib.pyplot as plt

import pickle
import numpy as np
import pandas as pd

def instances(fi):
    xseq = []
    yseq = []

    for line in fi:
        line = line.strip('\n')
        if not line:
            # An empty line means the end of a sentence.
            # Return accumulated sequences, and reinitialize.
            yield xseq, yseq
            xseq = []
            yseq = []
            continue

        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=form, 2=span_start, 3=span_end, 4=tag, 5...N = features
        item = fields[5:]
        xseq.append(item)

        # Append the label to the label sequence.
        yseq.append(fields[4])
    #return xseq, yseq

def load_data(fi):
    xtrain = []
    ytrain = []

    for line in fi:
        line = line.strip('\n')
        if not line:
            continue
        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=form, 2=span_start, 3=span_end, 4=tag, 5...N = features
        item = fields[5:]
        xtrain.append(item)

        # Append the label to the label sequence.
        ytrain.append(fields[4])
    return xtrain, ytrain

def instances_pred(fi):
    xseq = []
    toks = []

    for line in fi:
        line = line.strip('\n')
        if not line:
            # An empty line means the end of a sentence.
            # Return accumulated sequences, and reinitialize.
            yield xseq, toks
            xseq = []
            toks = []
            continue

        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=form, 2=span_start, 3=span_end, 4=tag, 5...N = features
        item = fields[5:]
        xseq.append(item)

        # Append the label to the label sequence.
        toks.append([fields[0],fields[1],fields[2],fields[3]])

In [103]:
sentences = []
tags = []

fi = open('../lstm-crf-glove/results/1/train.cod')

# Read sentences from STDIN, and append them to the trainer.
for xseq, yseq in instances(fi):
    sentences.append(xseq)
    tags.append(yseq)
fi.close()
fi = open('../lstm-crf-glove/results/1/train.cod')
(xtrain, ytrain) = load_data(fi)
x = pd.DataFrame(xtrain)
y = pd.DataFrame(ytrain)[0].values

max_len = 75
words = list(set(x[0].values))
words.append("ENDPAD")
n_words = len(words)
utags = list(set(y))
n_tags = len(utags)

word2idx = {w: i + 1 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
tag2idx = {t: i for i, t in enumerate(utags)}
tag2idx["PAD"] = 0

docs = [[w[0] for w in s] for s in sentences]

In [105]:
x[4]

0          DT
1          JJ
2          IN
3         VBN
4          NN
5         WRB
6          NN
7         VBZ
8         VBN
9          IN
10         JJ
11         JJ
12         NN
13         MD
14         VB
15        VBN
16          .
17         DT
18         NN
19         IN
20         NN
21         RB
22         IN
23        VBN
24         NN
25         NN
26         NN
27          (
28         NN
29         CC
         ... 
118334    VBZ
118335    VBN
118336     IN
118337     IN
118338     NN
118339    VBZ
118340    VBN
118341     IN
118342    NNS
118343     RB
118344    VBG
118345     NN
118346    NNS
118347      ,
118348     NN
118349    NNS
118350     MD
118351     VB
118352     RB
118353    VBD
118354     CC
118355     NN
118356     IN
118357     DT
118358     NN
118359     NN
118360     MD
118361     VB
118362     JJ
118363      .
Name: 4, Length: 118364, dtype: object

In [91]:
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, value=word2idx["PAD"], padding='post', truncating='post')

Y = [[tag2idx[t] for t in tag ]for tag in tags]
Y = pad_sequences(maxlen=max_len, sequences=Y, value=tag2idx["PAD"], padding='post', truncating='post')
#Y = [to_categorical(i, num_classes=n_tags) for i in Y]

In [72]:
max_len_char = 10
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

X_char = []
for sentence in sentences:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

80


In [92]:
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
             input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                   input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                        recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                       recurrent_dropout=0.6))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="softmax"))(main_lstm)
#crf = CRF(n_tags)  # CRF layer
#out = crf(model)  # output

model = Model([word_in, char_in], out)
#model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [93]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 75, 10)       0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 75)           0                                            
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, 75, 10, 10)   820         input_22[0][0]                   
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 75, 20)       161100      input_21[0][0]                   
__________________________________________________________________________________________________
time_distr

In [95]:
history = model.fit([X, np.array(X_char).reshape((len(X_char), max_len, max_len_char))],
                    np.array(Y).reshape(len(Y), max_len, 1), batch_size=32, epochs=5,
                validation_split=0.1, verbose=1)

hist = pd.DataFrame(history.history)

Train on 5107 samples, validate on 568 samples
Epoch 1/5
5107/5107 [==============================] - 21s 4ms/step - loss: 0.8059 - val_loss: 0.4236
Epoch 2/5
5107/5107 [==============================] - 17s 3ms/step - loss: 0.3404 - val_loss: 0.2546
Epoch 3/5
5107/5107 [==============================] - 17s 3ms/step - loss: 0.2115 - val_loss: 0.2026
Epoch 4/5
5107/5107 [==============================] - 17s 3ms/step - loss: 0.1582 - val_loss: 0.1789
Epoch 5/5
5107/5107 [==============================] - 17s 3ms/step - loss: 0.1267 - val_loss: 0.1760


In [97]:
for xseq,toks in instances_pred(open('../lstm-crf-glove/results/2/test.cod')):
    #x = pd.DataFrame(xseq)
    x_test_sent = [[word2idx.get(t[0],0) for t in xseq]]
    x_test_sent = pad_sequences(maxlen=max_len, sequences=x_test_sent, padding="post", value=n_words-1)
    
    sentence = [t[0] for t in xseq]
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)

    #prediction = model.predict(np.array([x_test_sent[0]]))[0]
    #model.predict([X_word_te,
    #                    np.array(X_char_te).reshape((len(X_char_te),
    #                                                 max_len, max_len_char))])
    x_test_sent = np.array([x_test_sent[0]])
    prediction = model.predict([x_test_sent, np.array(sent_seq).reshape((len(x_test_sent), max_len, max_len_char))])[0]
    prediction = np.argmax(prediction, axis=-1)
    inside = False;
    for k in range(0,len(toks)) :
        y = utags[prediction[k]]
        (sid, form, offS, offE) = toks[k]

        if (y[0]=="B") :
            entity_form = form
            entity_start = offS
            entity_end = offE
            entity_type = y[2:]
            inside = True
        elif (y[0]=="I" and inside) :
            entity_form += " "+form
            entity_end = offE
        elif (y[0]=="O" and inside) :
            #print(sid, entity_start+"-"+entity_end, entity_form, entity_type, sep="|")
            print(sid + "|" + entity_start+"-"+entity_end + "|" +entity_form + "|" +entity_type + "\n")
            inside = False

    if inside : print(sid + "|" + entity_start+"-"+entity_end + "|" + entity_form + "|" + entity_type + "\n")

DDI-DrugBank.d621.s1|26-35|colestipol|drug

DDI-DrugBank.d621.s1|38-51|cholestyramine|drug

DDI-DrugBank.d621.s1|71-80|sucralfate|drug

DDI-DrugBank.d672.s0|0-6|Digoxin|drug

DDI-DrugBank.d672.s0|63-69|digoxin|drug

DDI-DrugBank.d672.s1|55-66|theophylline|drug

DDI-DrugBank.d672.s2|35-46|theophylline|drug

DDI-DrugBank.d672.s2|96-103|caffeine|drug

DDI-DrugBank.d672.s3|0-7|Warfarin|drug

DDI-DrugBank.d672.s3|70-77|warfarin|drug

DDI-DrugBank.d672.s4|0-9|Cimetidine|drug

DDI-DrugBank.d672.s4|12-21|Cimetidine|drug

DDI-DrugBank.d672.s8|0-9|Probenecid|drug

DDI-DrugBank.d672.s8|12-21|Probenecid|drug

DDI-DrugBank.d733.s2|68-77|diclofenac|drug

DDI-DrugBank.d733.s2|82-90|ibuprofen|drug

DDI-DrugBank.d617.s1|0-8|lidocaine|drug

DDI-DrugBank.d617.s1|11-19|quinidine|drug

DDI-DrugBank.d617.s2|0-10|hydralazine|drug

DDI-DrugBank.d617.s3|11-19|dinitrate|drug

DDI-DrugBank.d617.s3|22-34|nitroglycerin|drug

DDI-DrugBank.d617.s4|0-13|chlorthalidone|drug

DDI-DrugBank.d617.s4|16-25|furosemide|drug


In [67]:
sentence

['It',
 'is',
 'possible',
 'that',
 'concomitant',
 'use',
 'of',
 'other',
 'known',
 'photosensitizing',
 'agents',
 'might',
 'increase',
 'the',
 'photosensitivity',
 'reaction',
 'of',
 'actinic',
 'keratoses',
 'treated',
 'with',
 'methyl',
 'aminolevulinate',
 'cream',
 '.']